# Regression Model in Keras
## Part of the Introduction to deep learning at Coursera

### 0. Import & Preprocessing Data - Split to train & test

In [2]:
import pandas as pd

#Import data
ctdf = pd.read_csv('data/concrete_data.csv')
ctdf.shape
ctdf.head()

#Extract predictors and target
ctPredColnames = ctdf.columns[ctdf.columns!='Strength']
ctPredictors = ctdf[ctPredColnames]
ctTarget = ctdf.drop(ctPredColnames, axis=1)

#Separate Train and Test data
from sklearn.model_selection import train_test_split
ctPredTrain, ctPredTest, ctTargetTrain, ctTargetTest = train_test_split(
    ctPredictors, ctTarget, test_size = 0.3, random_state = 8)

### A. Build a baseline model

In [3]:
#Import keras
from keras.models import Sequential
from keras.layers import Dense

#Create the model
def regression_model():
    model = Sequential()
    model.add(Dense(10, activation = 'relu', input_shape = (ctPredictors.shape[1],)))
    model.add(Dense(1))
    #Complie model
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')
    #Return
    return model

model = regression_model()

#Fit the model
fitted = model.fit(ctPredTrain, ctTargetTrain, 
                   validation_data = (ctPredTest, ctTargetTest),
                   epochs = 50, verbose = 0)

#Put loss into Dataframe
dfRep = pd.DataFrame(fitted.history)

#### Result of the baseline model

In [4]:
#Report sd, mean
print('(A)train (sd / mean):',round(dfRep['loss'].std(),2),'/',round(dfRep['loss'].mean(),2),
      '&','test (sd / mean):',round(dfRep['val_loss'].std(),2),'/',round(dfRep['val_loss'].mean(),2))

(A)train (sd / mean): 7076.09 / 1882.76 & test (sd / mean): 3680.25 / 1258.74


### B. Normalize the data

In [5]:
#Normalize predictors
ctPredTrainNorm = (ctPredTrain - ctPredTrain.mean())/ctPredTrain.std()
ctPredTestNorm = (ctPredTest - ctPredTest.mean())/ctPredTest.std()

#Fit model on normalized data
fittedNorm = model.fit(ctPredTrainNorm, ctTargetTrain, 
                   validation_data = (ctPredTestNorm, ctTargetTest),
                   epochs = 50, verbose = 0)

dfRepNorm = pd.DataFrame(fittedNorm.history)

#### Result of the model on normalized data

In [6]:
#Report sd, mean
print('(B)train (sd / mean):',round(dfRepNorm['loss'].std(),2),'/',round(dfRepNorm['loss'].mean(),2),
      '&','test (sd / mean):',round(dfRepNorm['val_loss'].std(),2),'/',round(dfRepNorm['val_loss'].mean(),2))

(B)train (sd / mean): 321.37 / 909.75 & test (sd / mean): 319.92 / 924.46


> We can see that the mean are drastically lower for B since the predictors have been normalized. Since mean squre error is a sum of error, it's natural to have lower sum with lower values.

### C. Increate the number of epochs

In [7]:
fittedNorm100 = model.fit(ctPredTrainNorm, ctTargetTrain, 
                   validation_data = (ctPredTestNorm, ctTargetTest),
                   epochs = 100, verbose = 0)

dfRepNorm100 = pd.DataFrame(fittedNorm100.history)

print('(C)train (sd / mean):',round(dfRepNorm100['loss'].std(),2),'/',round(dfRepNorm100['loss'].mean(),2),
      '&','test (sd / mean):',round(dfRepNorm100['val_loss'].std(),2),'/',round(dfRepNorm100['val_loss'].mean(),2))

(C)train (sd / mean): 91.44 / 238.14 & test (sd / mean): 92.51 / 252.85


> We can see that losses are lower as we increase epochs. This the model fits better as we iterate more. Hence, the losses gradually declined.

### D. Increase the number of hidden layers

In [8]:
def regression_model():
    model2 = Sequential()
    model2.add(Dense(10, activation = 'relu', input_shape = (ctPredictors.shape[1],)))
    model2.add(Dense(10, activation = 'relu'))
    model2.add(Dense(10, activation = 'relu'))
    model2.add(Dense(1))
    #Complie model
    model2.compile(optimizer = 'adam', loss = 'mean_squared_error')
    #Return
    return model2

model2 = regression_model()

fitted3layers = model2.fit(ctPredTrainNorm, ctTargetTrain, 
                   validation_data = (ctPredTestNorm, ctTargetTest),
                   epochs = 50, verbose = 0)

dfRepNorm3l = pd.DataFrame(fitted3layers.history)

print('(D)train (sd / mean):',round(dfRepNorm3l['loss'].std(),2),'/',round(dfRepNorm3l['loss'].mean(),2),
      '&','test (sd / mean):',round(dfRepNorm3l['val_loss'].std(),2),'/',round(dfRepNorm3l['val_loss'].mean(),2))

(D)train (sd / mean): 428.94 / 346.47 & test (sd / mean): 417.59 / 347.05


> The means of MSE are better than B but not better than C. This indicates that increasing hidden layers might result in a better model. However, that is not the only factor since C does better by only increasing epochs.